In [1]:
# Search Engine With Tools and agents

In [2]:
## Arxiv for research papers
## Tools Creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
# To run this query we need wrapper
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper


In [6]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(
    api_wrapper=api_wrapper_wiki,
)
wiki.name

'wikipedia'

In [8]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(
    api_wrapper=api_wrapper_arxiv,
)
print(arxiv.name)

arxiv


In [9]:
tools=[wiki,arxiv]
# Need model to run these and also chatpromptTemplate also

In [15]:
# Custom Tools [RAG Tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter 


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
import os
from dotenv import load_dotenv
load_dotenv()
# Set the environment variable for Hugging Face token
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [17]:
# Set HuggingFace Embeddings
embeddings=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\Gaurav\AppData\Local\Temp\ipykernel_24320\3788258163.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(


In [19]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents= text_splitter.split_documents(docs)
vectorstore=FAISS.from_documents(documents,embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022AAA643D30>, search_kwargs={})

In [20]:
# Convert retriever to tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="SmithDocsRetriever",
    description="A tool to retrieve documents from the Smith LangChain documentation.",
)
retriever_tool.name

'SmithDocsRetriever'

In [22]:
tools= [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Gaurav\\Desktop\\Gen Ai\\LangChain\\Learning 1\\myenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='SmithDocsRetriever', description='A tool to retrieve documents from the Smith LangChain documentation.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000022AAE66DB40>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_co

In [ ]:
## Run all this tools with Agent and LLM Models
## Tools, LLM--> AgentExeceutor
# Import Chatgroq
from langchain_groq import ChatGroq
# Set the Groq API key

groq_api_key = os.getenv("GROQ_API_KEY")
llm= ChatGroq(
    model="Llama3-8b-8192",
    api_key=groq_api_key)


In [29]:
## Prompt Template
# Just easy way to get
# But always use chat prompt template
# https://python.langchain.com/api_reference/langchain/agents/langchain.agents.openai_functions_agent.base.OpenAIFunctionsAgent.html

from langchain import hub # Load a prompt template from the hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [43]:
## Agents 
from langchain.agents import create_openai_tools_agent
agents=create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    
)
agents




RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [31]:
## To run the agent use agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=agents,
    tools=tools,
    llm=llm,
    verbose=True # Enable verbose mode to see the agent's reasoning and actions
)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [38]:
agent_executor.invoke({
    "input": "What is Langsmith?"
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of languagIt seems like you're looking for more information about Langsmith. Langsmith is actually another name for LangChain, which is a software framework that helps integrate large language models into applications.

> Finished chain.


{'input': 'What is Langsmith?',
 'output': "It seems like you're looking for more information about Langsmith. Langsmith is actually another name for LangChain, which is a software framework that helps integrate large language models into applications."}

In [39]:
agent_executor.invoke({
    "input": "What is Machine Learning?",
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Artificial Intelligence'}`


Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-makinMachine learning is a subset of artificial intelligence, and it's the part that focuses on developing and studying statistical algorithms that can learn from data. So, Machine learning is a way to implement AI in certain circumstances.

> Finished chain.


{'input': 'What is Machine Learning?',
 'output': "Machine learning is a subset of artificial intelligence, and it's the part that focuses on developing and studying statistical algorithms that can learn from data. So, Machine learning is a way to implement AI in certain circumstances."}

In [40]:
# We use this HUB because this is also recording history
agent_executor.invoke({
    "input": "What questions I asked before?",
})



> Entering new AgentExecutor chain...
This conversation has just started, so you haven't asked any questions yet.

> Finished chain.


{'input': 'What questions I asked before?',
 'output': "This conversation has just started, so you haven't asked any questions yet."}

In [42]:
agent_executor.invoke({
    "input": "What this research paper is about 1706.03762?",
    "chat_history": []
})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe research paper "1706.03762" is titled "Attention Is All You Need" and it's about a sequence transduction model that uses attention mechanisms.

> Finished chain.


{'input': 'What this research paper is about 1706.03762?',
 'chat_history': [],
 'output': 'The research paper "1706.03762" is titled "Attention Is All You Need" and it\'s about a sequence transduction model that uses attention mechanisms.'}